In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from openml import tasks, runs
from sklearn import neighbors
import math
import openml
from collections import Counter
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import datasets
from sklearn.metrics import accuracy_score

In [2]:
openml.config.apikey = '6a1d598d43fc357eb5b7b7afd49ab5f0'


In [31]:
dataset = openml.datasets.get_dataset(61)
dataset

OpenML Dataset
Name..........: iris
Version.......: 1
Format........: ARFF
Upload Date...: 2014-04-06 23:23:39
Licence.......: Public
Download URL..: https://api.openml.org/data/v1/download/61/iris.arff
OpenML URL....: https://www.openml.org/d/61
# of features.: 5
# of instances: 150

In [32]:
df, _, _, _ = dataset.get_data()
df.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [33]:
for index, row in df.iterrows():
    if row["class"] == "Iris-setosa":
        row["class"] = 0
    elif row["class"] == "Iris-versicolor":
        row["class"] = 2
    elif row["class"] == "Iris-virginica":
        row["class"] = 3
df.head()   

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [34]:
df["class"] = df["class"].replace(["Iris-setosa", "Iris-versicolor", "Iris-virginica"], [1, 2, 3])

df.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


In [35]:
X = df[["sepalwidth", "petallength", "petalwidth", "sepallength"]]
X.head()

,sepalwidth,petallength,petalwidth,sepallength
0,3.5,1.4,0.2,5.1
1,3.0,1.4,0.2,4.9
2,3.2,1.3,0.2,4.7
3,3.1,1.5,0.2,4.6
4,3.6,1.4,0.2,5.0


In [38]:
y = df["class"]
y = y.astype("int")
y.head()
y.mean()

2.0

In [39]:
class Node:
    """
    A class to represent a node in a decision tree.
    """
    def __init__(self, feature=None, value=None, left=None, right=None, outcome=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.outcome = outcome

class DecisionTree:
    """
    A class to represent a decision tree.
    """
    def __init__(self, max_depth=float("inf"), min_samples_split=2, min_impurity=1e-7):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity
        self.root = None

    def fit(self, X, y):
        data = pd.concat([X, y], axis=1)
        self.root = self._build_tree(data)

    def predict(self, X):
        return X.apply(lambda row: self._predict_row(row), axis=1)

    def _build_tree(self, data, depth=0):
        """
        Recursive function that builds the decision tree.
        """
        # Check if we have reached the maximum depth
        if depth == self.max_depth:
            return Node(outcome=self._most_common_outcome(data))

        # Check if we have reached a leaf node
        if len(data) < self.min_samples_split:
            return Node(outcome=self._most_common_outcome(data))

        # Check if the data is pure (all labels are the same)
        if self._entropy(data) < self.min_impurity:
            return Node(outcome=self._most_common_outcome(data))

        # Select the best feature to split the data
        best_feature, best_value = self._best_feature_to_split(data)

        # Split the data based on the best feature and value
        left_data, right_data = self._split_data(data, best_feature, best_value)

        # Recursively build the left and right subtrees
        left_subtree = self._build_tree(left_data, depth+1)
        right_subtree = self._build_tree(right_data, depth+1)

        # Create a new node to represent the best feature and value
        return Node(feature=best_feature, value=best_value, left=left_subtree, right=right_subtree)

    def _predict_row(self, row):
        """
        Recursive function that predicts the label of a single row.
        """
        node = self.root
        while node.outcome is None:
            if row[node.feature] == node.value:
                node = node.left
            else:
                node = node.right
        return node.outcome

    def _entropy(self, data):
        """
        Calculate the entropy of a set of data.
        """
        num_samples = len(data)
        value_counts = data.iloc[:, -1].value_counts()
        probabilities = value_counts / num_samples
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def _best_feature_to_split(self, data):
        """
        Select the best feature to split the data based on information gain.
        """
        best_gain = -1
        best_feature = None
        best_value = None
        entropy = self._entropy(data)

        for feature in data.columns[:-1]:
            values = data[feature].unique()

            for value in values:
                left_data, right_data = self._split_data(data, feature, value)

                if len(left_data) == 0:
                    continue
                if len(right_data) == 0:
                    continue

                gain = self._information_gain(data, left_data, right_data, entropy)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_value = value

        return best_feature, best_value

    def _information_gain(self, data, left_data, right_data, entropy):
        """
        Calculate the information gain from splitting the data into two groups.
        """
        p = len(left_data) / len(data)
        gain = entropy - p*self._entropy(left_data) - (1-p)*self._entropy(right_data)
        return gain

    def _split_data(self, data, feature, value):
        """
        Split the data based on a given feature and value.
        """
        left_data = data[data[feature] == value].reset_index(drop=True)
        right_data = data[data[feature] != value].reset_index(drop=True)
        return left_data, right_data

    def _most_common_outcome(self, data):
        """
        Return the most common outcome in the data.
        """
        outcome_counts = data.iloc[:, -1].value_counts()
        most_common_outcome = outcome_counts.index[0]
        return most_common_outcome


In [43]:
import numpy as np

class Node:
    """
    A class to represent a node in a decision tree.
    """
    def __init__(self, feature=None, value=None, left=None, right=None, outcome=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.outcome = outcome


class DecisionTree2:
    """
    A class to represent a decision tree.
    """
    def __init__(self, max_depth=float("inf"), min_samples_split=2, min_impurity=1e-7):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity
        self.root = None

    def fit(self, X, y):
        data = pd.concat([X, y], axis=1)
        self.root = self._build_tree(data)

    def predict(self, X):
        return X.apply(lambda row: self._predict_row(row), axis=1)

    def _build_tree(self, data, depth=0):
        """
        Recursive function that builds the decision tree.
        """
        # Check if we have reached the maximum depth
        if depth == self.max_depth:
            return Node(outcome=self._most_common_outcome(data))

        # Check if we have reached a leaf node
        if len(data) < self.min_samples_split:
            return Node(outcome=self._most_common_outcome(data))

        # Check if the data is pure (all labels are the same)
        if self._entropy(data) < self.min_impurity:
            return Node(outcome=self._most_common_outcome(data))

        # Select the best feature to split the data
        best_feature, best_value = self._best_feature_to_split(data)

        # Split the data based on the best feature and value
        left_data, right_data = self._split_data(data, best_feature, best_value)

        # Recursively build the left and right subtrees
        left_subtree = self._build_tree(left_data, depth+1)
        right_subtree = self._build_tree(right_data, depth+1)

        # Create a new node to represent the best feature and value
        return Node(feature=best_feature, value=best_value, left=left_subtree, right=right_subtree)

    def _predict_row(self, row):
        """
        Recursive function that predicts the label of a single row.
        """
        node = self.root
        while node.outcome is None:
            if row[node.feature] == node.value:
                node = node.left
            else:
                node = node.right
        return node.outcome

    def _mad(self, data):
        """
        Calculate the mean absolute deviation of a set of data.
        """
        target = data.iloc[:, -1]
        #print(target)
        mean = target.mean()
        mad = np.abs(target - mean).mean()
        return mad

    def _best_feature_to_split(self, data):
        """
        Select the best feature to split the data based on MAD reduction.
        """
        best_reduction = -1
        best_feature = None
        best_value = None

                # Calculate the MAD of the entire dataset
        mad_full = self._mad(data)

        # Loop through each feature and value
        for feature in data.columns[:-1]:
            for value in data[feature].unique():
                # Split the data based on the feature and value
                left_data, right_data = self._split_data(data, feature, value)

                # Skip if the split produced empty data
                if len(left_data) == 0 or len(right_data) == 0:
                    continue

                # Calculate the MAD reduction
                mad_left = self._mad(left_data)
                mad_right = self._mad(right_data)
                mad_reduction = mad_full - (len(left_data) / len(data) * mad_left) - (len(right_data) / len(data) * mad_right)

                # Update the best feature and value if the MAD reduction is higher
                if mad_reduction > best_reduction:
                    best_reduction = mad_reduction
                    best_feature = feature
                    best_value = value

        return best_feature, best_value

    def _split_data(self, data, feature, value):
        """
        Split the data based on a feature and value.
        """
        left_data = data[data[feature] == value]
        right_data = data[data[feature] != value]
        return left_data, right_data

    def _most_common_outcome(self, data):
        """
        Return the most common outcome in the data.
        """
        return data.iloc[:, -1].mode()[0]

    def _entropy(self, data):
        """
        Calculate the entropy of a set of data.
        """
        target = data.iloc[:, -1]
        p = target.value_counts(normalize=True)
        entropy = -(p * np.log2(p)).sum()
        return entropy

    def print_tree(self, node=None, depth=0):
        """
        Print the decision tree in a readable format.
        """
        if node is None:
            node = self.root

        if node.outcome is not None:
            print("    " * depth + "Outcome:", node.outcome)
        else:
            print("    " * depth + "Feature:", node.feature)
            print("    " * depth + "Value:", node.value)
            self.print_tree(node.left, depth+1)
            self.print_tree(node.right, depth+1)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [45]:
clf = DecisionTree2(max_depth=5)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)

In [46]:
accuracy_score(y_test, predictions)

0.6666666666666666